# ColumnTransformer: 여러 컬럼에 각기 다른 전처리 적용하기

이 노트북은 `scikit-learn`의 `ColumnTransformer` 클래스를 사용하여 데이터프레임의 각기 다른 컬럼에 서로 다른 전처리 방법을 한 번에 적용하는 효율적인 방법을 다룹니다.

실제 데이터셋은 숫자형, 범주형 등 다양한 타입의 특성이 섞여 있는 경우가 많습니다. 이럴 때, 우리는 보통 다음과 같은 전처리를 수행합니다.
- **숫자형 특성**: 값의 범위를 조정하는 스케일링 (예: `StandardScaler`)
- **범주형 특성**: 문자열을 숫자 형태로 변환하는 인코딩 (예: `OneHotEncoder`)

`ColumnTransformer`는 이러한 작업을 하나의 단계로 묶어 코드를 간결하고 체계적으로 만들어 줍니다.

### 1. 라이브러리 임포트 및 데이터 로드

In [ ]:
import pandas as pd
import mglearn
import os
import numpy as np

# 전처리를 위한 scikit-learn 클래스
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
# adult 데이터셋 로드
file_path = os.path.join(mglearn.datasets.DATA_PATH, "adult.data")
data_all = pd.read_csv(file_path, header=None, index_col=False, 
                   names=['age', 'workclass', 'fnlwgt', 'education',  'education-num',
                          'marital-status', 'occupation', 'relationship', 'race', 'gender',
                          'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
                          'income'])

# 분석에 사용할 특성만 선택
data = data_all[['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']]

print("원본 데이터 샘플:")
data.head()

### 2. ColumnTransformer 설정

`ColumnTransformer`를 설정하여 어떤 컬럼에 어떤 변환을 적용할지 정의합니다.

- **`age`, `hours-per-week`** (연속형 숫자) 컬럼에는 `StandardScaler`를 적용하여 표준 정규분포를 따르도록 스케일링합니다.
- **`workclass`, `education`, `gender`, `occupation`** (범주형 문자) 컬럼에는 `OneHotEncoder`를 적용하여 원-핫 인코딩을 수행합니다.

In [ ]:
# ColumnTransformer 객체 생성
# 리스트 형태로 (이름, 변환기, 적용할 컬럼 리스트) 튜플을 전달합니다.
ct = ColumnTransformer([
    ("scaling", StandardScaler(), ['age', 'hours-per-week']),
    ("onehot", OneHotEncoder(sparse_output=False), ['workclass', 'education', 'gender', 'occupation'])
    # income 컬럼은 타겟 변수이므로 여기서는 제외합니다.
])
# remainder='passthrough' 옵션을 사용하면 지정되지 않은 컬럼을 그대로 유지할 수 있습니다.

### 3. 데이터 변환 (Fit and Transform)

설정한 `ColumnTransformer`를 데이터에 적용합니다.
- `fit()`: 데이터로부터 스케일링에 필요한 평균/표준편차, 인코딩에 필요한 카테고리 등을 학습합니다.
- `transform()`: 학습된 정보를 바탕으로 실제 데이터를 변환합니다.
- `fit_transform()`: 위 두 과정을 한 번에 수행합니다.

In [ ]:
# 타겟 변수를 제외한 데이터에 ColumnTransformer를 적용합니다.
# income 컬럼은 예측 대상이므로 입력 데이터에서 제외합니다.
data_features = data.drop("income", axis=1)

# fit_transform을 사용하여 학습과 변환을 동시에 수행
transformed_data = ct.fit_transform(data_features)

print("변환된 데이터의 형태:", transformed_data.shape)
print("
변환된 데이터 (Numpy 배열) 샘플:")
print(transformed_data[:5, :])

### 4. 변환된 데이터 확인 (심화)

`ColumnTransformer`의 결과는 컬럼 이름이 없는 `Numpy` 배열입니다. `get_feature_names_out()` 메소드를 사용하면 변환 후 생성된 특성들의 이름을 확인할 수 있어, 결과를 해석하는 데 도움이 됩니다.

In [ ]:
# 변환 후의 특성 이름 가져오기
feature_names = ct.get_feature_names_out()
print("생성된 특성 이름 (처음 10개):", feature_names[:10])

# 변환된 데이터를 다시 DataFrame으로 만들어 가독성 높이기
transformed_df = pd.DataFrame(transformed_data, columns=feature_names)

print("
변환된 데이터를 DataFrame으로 표시:")
transformed_df.head()